modified from the unbond-generics example on hackage

On first run `stack build` will install dependencies specified in the cabl file into the `pwd` directory,
but import will fail.
Then, restart the kernel and run this notebook again.
On the second run after restart, the `stack build` won't do anyting because the build is up to date
and the newly loaded kernel should be able to see the newly installed modules. 

In [3]:
:!stack build

In [6]:
:!stack install unbound-generics

In [1]:
:!stack exec ghc-pkg -- list

/opt/stack/programs/x86_64-linux/ghc-8.6.5/lib/ghc-8.6.5/package.conf.d
    Cabal-2.4.0.1
    array-0.5.3.0
    base-4.12.0.0
    binary-0.8.6.0
    bytestring-0.10.8.2
    containers-0.6.0.1
    deepseq-1.4.4.0
    directory-1.3.3.0
    filepath-1.4.2.1
    (ghc-8.6.5)
    ghc-boot-8.6.5
    ghc-boot-th-8.6.5
    ghc-compact-0.1.0.0
    ghc-heap-8.6.5
    ghc-prim-0.5.3
    ghci-8.6.5
    haskeline-0.7.4.3
    hpc-0.6.0.3
    integer-gmp-1.0.2.0
    libiserv-8.6.3
    mtl-2.2.2
    parsec-3.1.13.0
    pretty-1.1.3.6
    process-1.6.5.0
    rts-1.0
    stm-2.5.0.0
    template-haskell-2.14.0.0
    terminfo-0.4.1.2
    text-1.2.3.1
    time-1.8.0.2
    transformers-0.5.6.2
    unix-2.7.2.2
    xhtml-3000.2.2.1

/opt/stack/snapshots/x86_64-linux/lts-13.26/8.6.5/pkgdb
    Cabal-2.4.1.0
    JuicyPixels-3.3.3
    StateVar-1.1.1.1
    aeson-1.4.2.0
    aeson-pretty-0.8.7
    ansi-terminal-0.8.2
    ansi-wl-pprint-0.6.8.2
    asn1-encoding-0.9.5
    asn1-parse-0.9.4
    asn1-types-0.3.2
    a

In [2]:
{-# LANGUAGE DeriveDataTypeable
           , DeriveGeneric
           , MultiParamTypeClasses #-}

import Unbound.Generics.LocallyNameless
import GHC.Generics (Generic)
import Data.Typeable (Typeable)

In [3]:
-- | Names for expressions
type Nm = Name Exp

-- | Expressions
data Exp = Var Nm            -- ^ variables
         | Lam (Bind Nm Exp) -- ^ lambdas bind a variable within a body expression
         | App Exp Exp       -- ^ application
          deriving (Show, Generic, Typeable)

-- Automatically construct alpha equiv, free vars, and binding operations.
instance Alpha Exp

-- semi-automatically implement capture avoiding substitution
instance Subst Exp Exp where
  -- `isvar` identifies the variable case in your AST.
  isvar (Var x) = Just (SubstName x)
  isvar _       = Nothing

In [4]:
-- call-by-value evaluation
-- evaluation takes an expression and returns a value while using a source of fresh names
eval :: Exp -> FreshM Exp
eval (Var x)     = fail $ "unbound variable " ++ show x
eval e@(Lam {})  = return e
eval (App e1 e2) = do
  v1 <- eval e1
  v2 <- eval e2
  case v1 of
   (Lam bnd) -> do
     -- open the lambda by picking a fresh name for the bound variable x in body
     (x, body) <- unbind bnd
     eval (subst x v2 body)
   _ -> fail "application of non-lambda"

In [5]:
x = s2n "x"
y = s2n "y"
e = Lam $ bind x (Lam $ bind y (App (Var y) (Var x)))

In [6]:
e

Lam (<x> Lam (<y> App (Var 0@0) (Var 1@0)))

In [7]:
runFreshM $ eval (App (App e e) e)

Lam (<y> App (Var 0@0) (Lam (<x> Lam (<y> App (Var 0@0) (Var 1@0)))))